In [31]:
!pip install dagshub mlflow --quiet

import warnings
warnings.filterwarnings("ignore")

print("Done!")

Done!


In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [33]:
import mlflow.sklearn
from datetime import datetime
import joblib
import dagshub
import mlflow
from sklearn.base import BaseEstimator, TransformerMixin

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

Initialized MLflow to track repo "gnada22/ml_final_project"

Repository gnada22/ml_final_project initialized!

In [37]:
class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["weekofyear"] = X["Date"].dt.isocalendar().week.astype(int)
        X["sin_13"] = np.sin(2 * np.pi * X["weekofyear"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["weekofyear"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["weekofyear"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["weekofyear"] / 23)
        X = X.drop(columns=["Date"])
        return X

local_path = mlflow.artifacts.download_artifacts(
    artifact_uri="mlflow-artifacts:/bdaee488c065473c96d054e5ae43539a/abb40eeefeb54f7a87763486cb01f016/artifacts/model.pkl"
)

model = joblib.load(local_path)

print("Done!")

Done!


In [38]:
# load and add custom features

test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip", parse_dates=["Date"])
features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = test.merge(features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(stores, on="Store", how="left")

def add_lag_features(df):
    train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
    train = train[["Store", "Dept", "Date", "Weekly_Sales"]]
    train = train.sort_values(["Store", "Dept", "Date"])
    
    full = pd.concat([train, df], axis=0)
    full = full.sort_values(["Store", "Dept", "Date"])
    
    full["lag_1"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
    full["lag_52"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)

    res = full[full["Weekly_Sales"].isna()].copy() # just test rows
    # res.dropna(subset=["lag_1", "lag_52"], inplace=True)  # drop rows with missing lags

    return res

df = add_lag_features(df)

X_test = df.copy()

In [40]:
preds = model.predict(X_test)
df["Weekly_Sales"] = preds

df["Id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str) + "_" + df["Date"].dt.strftime("%Y-%m-%d")
submission = df[["Id", "Weekly_Sales"]]
submission.to_csv("submission.csv", index=False)
print("✅ Submission saved as submission.csv")

✅ Submission saved as submission.csv
